In [1]:
from xcube.core.store import new_data_store

store = new_data_store("s3", root='agriculture-vlab-data-staging', max_depth=5, storage_options=dict(anon=True))

In [2]:
# list(store.get_data_ids())

In [3]:
cube = store.open_data('avl/l3b/2020/bel/S2_L3B_LAI_31UFS.zarr')
cube

,Array,Chunk
Bytes,65.12 GiB,4.00 MiB
Shape,"(145, 10980, 10980)","(1, 1024, 1024)"
Count,17546 Tasks,17545 Chunks
Type,float32,numpy.ndarray


In [4]:
from avl.verify import verify_dataset

In [5]:
verify_dataset(cube)

[('WARNING', "missing global attribute 'history'"),
 ('WARNING', "missing global attribute 'time_coverage_end'"),
 ('WARNING', "missing global attribute 'geospatial_lon_min'"),
 ('WARNING', "missing global attribute 'geospatial_lon_max'"),
 ('WARNING', "missing global attribute 'geospatial_lon_resolution'"),
 ('WARNING', "missing global attribute 'geospatial_lon_units'"),
 ('WARNING', "missing global attribute 'geospatial_lat_min'"),
 ('WARNING', "missing global attribute 'geospatial_lat_max'"),
 ('WARNING', "missing global attribute 'geospatial_lat_resolution'"),
 ('WARNING', "missing global attribute 'geospatial_lat_units'"),
 ('WARNING', "missing attribute 'long_name' in variable 'time'"),
 ('WARNING', "missing attribute 'units' in variable 'time'"),
 ('WARNING', "missing attribute 'long_name' in variable 'crs'"),
 ('WARNING', "missing attribute 'units' in variable 'crs'")]

In [6]:
cube.attrs

{'Conventions': 'CF-1.7',
 'geospatial_bounds': 'POLYGON((600000.0 5700000.0, 600000.0 5590200.0, 709800.0 5590200.0, 709800.0 5700000.0, 600000.0 5700000.0))',
 'geospatial_bounds_crs': 'PROJCRS["WGS 84 / UTM zone 31N",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 31N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",3,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing",n